# cat_col 상관관계 분석

In [3]:
import pandas as pd
import numpy as np

train_df = pd.read_csv(r'C:\A\breastCancer\ml/train.csv')
test_df = pd.read_csv(r"C:\A\breastCancer\ml/test.csv")


In [4]:
import os
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if not IN_COLAB:
    os.chdir("..")
from sklearn.datasets import fetch_covtype
import random
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
%load_ext autoreload
%autoreload 2


In [18]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
import os
import warnings
warnings.filterwarnings("ignore")


In [19]:
# median
train_df['size of cancers'] = train_df['size of cancers'].fillna(train_df['size of cancers'].median())
train_df = train_df.fillna(0)

test_df['size of cancers'] = test_df['size of cancers'].fillna(train_df['size of cancers'].median())
test_df = test_df.fillna(0)


target_name = ['N_category']
# 명목형 변수
cat_col_names = [
    'dx','location of cancers', 'number of cancers', 'NG','HG','HG_score_1','HG_score_2','HG_score_3','DCIS_or_LCIS','DCIS_or_LCIS_type','T_category','ER','PR','HER2','HER2_IHC','HER2_SISH',"BRCA_mutation"
]

for col_name in cat_col_names:
    cat_values = np.unique(train_df[col_name])
    
    #print(f"{col_name}: {cat_values}\n")


# 수치형 변수
num_col_names = [
    'age','size of cancers','ER_Allred_score','PR_Allred_score','KI-67_LI_percent','HER2_SISH_ratio'
]

feature_columns = (
    num_col_names + cat_col_names + target_name)

train_df = train_df.loc[:,feature_columns]


test = test_df.loc[:,num_col_names + cat_col_names]


X = train_df.loc[:,cat_col_names]
y = train_df.loc[:,target_name]


In [20]:
import re
df = train_df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                1000 non-null   int64  
 1   sizeofcancers      1000 non-null   float64
 2   ER_Allred_score    1000 non-null   float64
 3   PR_Allred_score    1000 non-null   float64
 4   KI67_LI_percent    1000 non-null   float64
 5   HER2_SISH_ratio    1000 non-null   float64
 6   dx                 1000 non-null   int64  
 7   locationofcancers  1000 non-null   int64  
 8   numberofcancers    1000 non-null   int64  
 9   NG                 1000 non-null   float64
 10  HG                 1000 non-null   float64
 11  HG_score_1         1000 non-null   float64
 12  HG_score_2         1000 non-null   float64
 13  HG_score_3         1000 non-null   float64
 14  DCIS_or_LCIS       1000 non-null   int64  
 15  DCIS_or_LCIS_type  1000 non-null   float64
 16  T_category         1000 n

In [21]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   dx                   1000 non-null   int64  
 1   location of cancers  1000 non-null   int64  
 2   number of cancers    1000 non-null   int64  
 3   NG                   1000 non-null   float64
 4   HG                   1000 non-null   float64
 5   HG_score_1           1000 non-null   float64
 6   HG_score_2           1000 non-null   float64
 7   HG_score_3           1000 non-null   float64
 8   DCIS_or_LCIS         1000 non-null   int64  
 9   DCIS_or_LCIS_type    1000 non-null   float64
 10  T_category           1000 non-null   float64
 11  ER                   1000 non-null   float64
 12  PR                   1000 non-null   float64
 13  HER2                 1000 non-null   float64
 14  HER2_IHC             1000 non-null   float64
 15  HER2_SISH            1000 non-null   fl

In [22]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   N_category  1000 non-null   int64
dtypes: int64(1)
memory usage: 7.9 KB


In [23]:
import pandas as pd
from scipy.stats import chi2_contingency

dic_result = {
    '독립변수':[],
    '카이제곱 통계량':[],
    'p-value':[],
    'df':[],
    '기댓값 행렬':[]
}
df_expected = pd.DataFrame()

for i in X.columns:
  dic_result['독립변수'].append(i)
  A = np.array(X[i])
  B = np.array(y['N_category'])

  #crosstab 객체의 margins=False여야 
  crosstab=pd.crosstab(A, B, margins=False)
  # 교차분석 결과값들 일괄계산
  # correction=True를 적용하면 Yates' correction이 적용 되어 검정통계량이 보수적으로 더 낮게 나옴
  result=chi2_contingency(observed=crosstab, correction=False)
  dic_result['카이제곱 통계량'].append(result[0])
  dic_result['p-value'].append(result[1])
  dic_result['df'].append(result[2])
  dic_result['기댓값 행렬'].append(result[3])
  df= pd.DataFrame(result[3]).rename(index={0:i}, columns={0:'림프절전이X', 1:'림프절전이O'})
  df_expected= pd.concat([df_expected,df])

df_result = pd.DataFrame(dic_result)  

In [24]:
df_result

,독립변수,카이제곱 통계량,p-value,df,기댓값 행렬
0,dx,10.369261,1.567452e-02,3,"[[416.502, 440.498], [31.59, 33.41], [4.86, 5...."
1,location of cancers,0.910054,6.344309e-01,2,"[[229.392, 242.608], [240.57, 254.43], [16.038..."
2,number of cancers,72.307456,1.841503e-17,1,"[[421.362, 445.638], [64.638, 68.362]]"
3,NG,195.690607,3.599583e-42,3,"[[24.786, 26.214], [95.256, 100.744], [238.14,..."
4,HG,150.512829,2.042389e-32,3,"[[41.796, 44.204], [138.024, 145.976], [213.84..."
5,HG_score_1,144.208756,4.676095e-31,3,"[[44.712, 47.288], [40.824, 43.176], [100.116,..."
6,HG_score_2,184.104254,1.145727e-39,3,"[[44.712, 47.288], [67.554, 71.446], [242.514,..."
7,HG_score_3,81.499503,8.380342e-17,4,"[[43.254, 45.746], [310.554, 328.446], [76.302..."
8,DCIS_or_LCIS,185.077813,6.469393e-41,2,"[[291.114, 307.886], [40.338, 42.662], [154.54..."
9,DCIS_or_LCIS_type,44.688106,1.977429e-10,2,"[[424.764, 449.236], [33.534, 35.466], [27.702..."


In [15]:
df_result.to_csv(r'C:\A\breastCancer\ml/output/coll/df_result_cat_corrleation_2.csv')

In [16]:
df_expected

,림프절전이X,림프절전이O
dx,416.502,440.498
1,31.590,33.410
2,4.860,5.140
3,33.048,34.952
location of cancers,229.392,242.608
1,240.570,254.430
2,16.038,16.962
number of cancers,421.362,445.638
1,64.638,68.362
NG,24.786,26.214


In [17]:
df_expected.to_csv(r'C:\A\breastCancer\ml\output\coll/df_expected_cat_corrleation_2.csv')

# num_col 상관관계분석

In [25]:
import os
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if not IN_COLAB:
    os.chdir("..")
from sklearn.datasets import fetch_covtype
import random
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
import os
import warnings
warnings.filterwarnings("ignore")


In [27]:
import pandas as pd
import numpy as np

train_df = pd.read_csv(r'C:\A\breastCancer\ml/train.csv')
test_df = pd.read_csv(r"C:\A\breastCancer\ml/test.csv")


In [28]:
# median
train_df['size of cancers'] = train_df['size of cancers'].fillna(train_df['size of cancers'].median())
train_df = train_df.fillna(0)

test_df['size of cancers'] = test_df['size of cancers'].fillna(train_df['size of cancers'].median())
test_df = test_df.fillna(0)


target_name = ['N_category']
# 명목형 변수
cat_col_names = [
    'dx','location of cancers', 'number of cancers', 'NG','HG','HG_score_1','HG_score_2','HG_score_3','DCIS_or_LCIS','DCIS_or_LCIS_type','T_category','ER','PR','HER2','HER2_IHC','HER2_SISH',"BRCA_mutation"
]

for col_name in cat_col_names:
    cat_values = np.unique(train_df[col_name])
    
    #print(f"{col_name}: {cat_values}\n")


# 수치형 변수
num_col_names = [
    'age','size of cancers','ER_Allred_score','PR_Allred_score','KI-67_LI_percent','HER2_SISH_ratio'
]

feature_columns = (
    num_col_names + cat_col_names + target_name)

train_df = train_df.loc[:,feature_columns]


test = test_df.loc[:,num_col_names + cat_col_names]


X = train_df.loc[:,num_col_names]
y = train_df.loc[:,target_name]


In [29]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               1000 non-null   int64  
 1   size of cancers   1000 non-null   float64
 2   ER_Allred_score   1000 non-null   float64
 3   PR_Allred_score   1000 non-null   float64
 4   KI-67_LI_percent  1000 non-null   float64
 5   HER2_SISH_ratio   1000 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 47.0 KB


In [30]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   N_category  1000 non-null   int64
dtypes: int64(1)
memory usage: 7.9 KB


In [31]:
import pandas as pd
import scipy.stats as stats

dic_result = {
    '독립변수':[],
    'correlation':[],
    'p-value':[]
}
df_expected = pd.DataFrame()

for i in X.columns:
  dic_result['독립변수'].append(i)
  A = (X[i])
  B = y['N_category']
  result = stats.pearsonr(A,B)
  dic_result['correlation'].append(result[0])
  dic_result['p-value'].append(result[1])

df_result = pd.DataFrame(dic_result)  

In [32]:
df_result

,독립변수,correlation,p-value
0,age,0.069017,2.908133e-02
1,size of cancers,0.287994,1.494569e-20
2,ER_Allred_score,0.106277,7.623583e-04
3,PR_Allred_score,0.072370,2.209751e-02
4,KI-67_LI_percent,0.210824,1.646682e-11
5,HER2_SISH_ratio,0.069395,2.820974e-02


In [23]:
df_result.to_csv(r'C:\A\breastCancer\ml\output\coll/df_result_num_corrleation_2.csv')

# 분류 시작

In [ ]:
SEED = 88
NSPLIT = 10

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(SEED) # Seed 고정

In [ ]:
"""
# split the dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = SEED)
"""

from sklearn.model_selection import KFold

X = np.array(X)                 #np.array(df.iloc[:, :-1]) # class 열 제외한 feature 열들 모음 -> array 변환
y = np.array(y)                           #df['class']



In [ ]:
# split 개수, 셔플 여부 및 seed 설정
kf = KFold(n_splits = NSPLIT, shuffle = True, random_state = SEED)

# split 개수 스텝 만큼 train, test 데이터셋을 매번 분할
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

## lightGBM

In [ ]:
import lightgbm as lgb

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

accuracy_history = []
f1_score_history = []

# K-fold 검증 과정으로 실제 랜덤 포레스트 모델을 학습하여 정확도 평균을 내는 방법
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    
    clf = lgb.LGBMClassifier(n_estimators=5, random_state=SEED) # 모델 선언
    clf.fit(X_train,y_train) # 모델 학습
    y_pred=clf.predict(X_test) # 예측 라벨
    accuracy_history.append(accuracy_score(y_pred, y_test)) # 정확도 측정 및 기록
    f1_score_history.append(f1_score(y_pred, y_test)) # 정확도 측정 및 기록

print("accuracy 각 분할의 정확도 기록 :", accuracy_history)
print("accuracy 평균 정확도 :", np.mean(accuracy_history))

print("f1_score 각 분할의 정확도 기록 :", f1_score_history)
print("f1_score 평균 정확도 :", np.mean(f1_score_history))

accuracy 각 분할의 정확도 기록 : [0.85, 0.79, 0.73, 0.77, 0.75, 0.71, 0.77, 0.77, 0.75, 0.79]
accuracy 평균 정확도 : 0.768
f1_score 각 분할의 정확도 기록 : [0.8421052631578948, 0.8073394495412846, 0.7157894736842104, 0.7722772277227723, 0.7252747252747253, 0.7184466019417476, 0.8067226890756303, 0.7766990291262137, 0.7368421052631577, 0.817391304347826]
f1_score 평균 정확도 : 0.7718887869135462


## catboost classififier

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

accuracy_history = []
f1_score_history = []

# K-fold 검증 과정으로 실제 랜덤 포레스트 모델을 학습하여 정확도 평균을 내는 방법
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf = CatBoostClassifier(
      iterations=5, 
      learning_rate=0.1, 
      loss_function='CrossEntropy',
      random_state=SEED) # 모델 선언

    clf.fit(X_train,y_train) # 모델 학습
    y_pred=clf.predict(X_test) # 예측 라벨
    accuracy_history.append(accuracy_score(y_pred, y_test)) # 정확도 측정 및 기록
    f1_score_history.append(f1_score(y_pred, y_test)) # 정확도 측정 및 기록

print("accuracy 각 분할의 정확도 기록 :", accuracy_history)
print("accuracy 평균 정확도 :", np.mean(accuracy_history))

print("f1_score 각 분할의 정확도 기록 :", f1_score_history)
print("f1_score 평균 정확도 :", np.mean(f1_score_history))

## test preidiction

In [ ]:
test_df_id = test_df.loc[:,'ID']
len(test_df_id),type(test_df_id)

(250, pandas.core.series.Series)

In [ ]:
array = np.array(test_df_id.values)
len(array),type(array)

(250, numpy.ndarray)

In [ ]:
test_pred = clf.predict(test)
len(test_pred),type(test_pred)

(250, numpy.ndarray)

In [ ]:
dic_pred={
    'ID' : array,
    'N_category':test_pred
}

In [ ]:
df_test_pred = pd.DataFrame(dic_pred)

In [ ]:
df_test_pred

,ID,N_category
0,BC_01_0011,1
1,BC_01_0220,1
2,BC_01_0233,0
3,BC_01_0258,0
4,BC_01_0260,1
...,...,...
245,BC_01_3328,1
246,BC_01_3404,0
247,BC_01_3418,0
248,BC_01_3438,0


In [ ]:
df_test_pred.to_csv(f'/content/drive/MyDrive/project/BreastCancer/ml/output/lgbm_{SEED}_{f1score}.csv')

In [ ]:
"""import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf
clf.fit(X_train,y_train)
# predict the results
y_pred=clf.predict(X_test)
# view accuracy
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))
y_pred_train = clf.predict(X_train)
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))


# print the scores on training and test set
print('Training set score: {:.4f}'.format(clf.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(X_test, y_test)))
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
f1score=f1_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))
print('LightGBM Model f1score score: {0:0.4f}'.format(f1_score(y_pred, y_test)))"""

In [ ]:
"""from catboost import CatBoostClassifier
clf = CatBoostClassifier(
    iterations=5, 
    learning_rate=0.1, 
    loss_function='CrossEntropy'
)
clf.fit(X_train,y_train)
# predict the results
y_pred=clf.predict(X_test)
# view accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('catboost classififier Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))
y_pred_train = clf.predict(X_train)
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))
# print the scores on training and test set

print('Training set score: {:.4f}'.format(clf.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(X_test, y_test)))
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('catboost classififier Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))"""